# Intermediate Processing
- group overlapping dataframes into a single dataframe
- runtime: \< 30m

In [1]:
import os
import numpy as np
import pandas as pd
from collections import defaultdict
from pathlib import Path
from tqdm import tqdm

/tmp/ipykernel_6418/3142606034.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
COLS_VA = ['TA_F', 'SW_IN_F', 'LW_IN_F', 'VPD_F', 'PA_F', 'P_F', 'WS_F', 'WD', 'RH', 'USTAR', 'NETRAD', 'PPFD_IN', 'PPFD_DIF', 'PPFD_OUT', 'SW_DIF', 'SW_OUT', 'LW_OUT',
        'CO2_F_MDS', 'G_F_MDS', 'LE_F_MDS', 'H_F_MDS', 'NEE_VUT_REF', 'RECO_NT_VUT_REF', 'RECO_DT_VUT_REF', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF']
COLS_QC = [f'{c}_QC' for c in COLS_VA]
COLS_TS = ['TIMESTAMP_START']

INTERMEDIATE_DIR = Path('data/intermediate/final_int_1')
META_DIR = Path('data/meta')
OUTPUT_DIR = Path('data/intermediate/final_int_2')

## Part 2: Combining observations from different datasets

In [3]:
PRIORITIES = {
    'ameriflux': 1,
    'icos-2023': 1,
    'icos-ww': 2,
    'fluxnet': 3
}

def add_hour(timestamp):
    year = timestamp[0:4]
    month = timestamp[4:6]
    day = timestamp[6:8]
    hour = timestamp[8:]
    if month == '12' and day == '31':
        return f'{str(int(year)+1)}01010000'
    else:
        return timestamp


def has_overlap(data_1, data_2):
    start_1 = str(data_1[1])
    end_1 = add_hour(str(data_1[2]))
    start_2 = str(data_2[1])
    end_2 = add_hour(str(data_2[2]))
    disjoint_before = end_1 < start_2
    disjoint_after = start_1 > end_2
    return not disjoint_before and not disjoint_after


def merge_sites(data_1, data_2):
    # implement
    df1 = data_1[0]
    df2 = data_2[0]
    merged_df = pd.merge(df1, df2, on='TIMESTAMP_START', how='outer', suffixes=('_df1', '_df2'))

    # If QC value is better in one than the other, set null.
    for col in COLS_VA:
        c1 = f'{col}_df1'
        c2 = f'{col}_df2'
        qc1 = f'{col}_QC_df1'
        qc2 = f'{col}_QC_df2'

        merged_df[qc1] = merged_df[qc1].fillna(5)
        merged_df[qc2] = merged_df[qc2].fillna(5)
        qcidx = merged_df[qc1]-merged_df[qc2]
        # at this point, qcidx <= 0 means keep df1 value. qcidx > 0 means keep df2 value.

        merged_df.loc[qcidx > 0.0, c1] = np.nan
        merged_df[col] = merged_df[c1].combine_first(merged_df[c2])
        merged_df[f'{col}_QC'] = merged_df[[qc1, qc2]].min(axis=1)
        merged_df.loc[merged_df[f'{col}_QC'] == 5, f'{col}_QC'] = np.nan
        merged_df = merged_df.drop(labels=[c1, c2, qc1, qc2], axis=1).sort_values(by='TIMESTAMP_START')
    return (merged_df, merged_df.index[0], merged_df.index[-1].max(), f'{data_1[-1]},{data_2[-1]}')


def merge_site_data(site_data):
    all_merged = False
    while not all_merged:
        merged_sites = []
        all_merged = True
        i = 0
        while i < len(site_data):
            if i == len(site_data)-1:
                merged_sites.append(site_data[i])
                i += 1
            elif has_overlap(site_data[i], site_data[i+1]):
                all_merged = False
                new_data = merge_sites(site_data[i], site_data[i+1])
                merged_sites.append(new_data)
                i += 2
            else:
                merged_sites.append(site_data[i])
                i += 1

        site_data = merged_sites
    return site_data


def process_unmerged_site_data(site, csv_info):
    site_out_dir = os.path.join(OUTPUT_DIR, site)
    if not os.path.exists(site_out_dir):
        os.makedirs(site_out_dir)
    
    files = [file for _, file in csv_info]
    site_data = []
    for file in files:
        df = pd.read_csv(file)
        df = df.set_index('TIMESTAMP_START')
        filename = file.stem
        start, end, source = filename.split('_')
        start = int(start)
        end = int(end)
        site_data.append((df, start, end, source))
    site_data = sorted(site_data, key=lambda x: PRIORITIES[x[-1]])
    
    if len(site_data) > 1:
        site_data = merge_site_data(site_data)

    for site in site_data:
        if not os.path.exists(os.path.join(site_out_dir, f'{site[1]}_{site[2]}_{site[3]}')):
            os.makedirs(os.path.join(site_out_dir, f'{site[1]}_{site[2]}_{site[3]}'))
        site[0].to_csv(os.path.join(site_out_dir, f'{site[1]}_{site[2]}_{site[3]}', 'data.csv'))



In [4]:
sites = defaultdict(list)
for source in os.listdir(INTERMEDIATE_DIR):
    for site in os.listdir(INTERMEDIATE_DIR / source):
        if site == 'site_data.csv':
            continue
        site_file = os.listdir(INTERMEDIATE_DIR / source / site)[0]
        sites[site].append((source, INTERMEDIATE_DIR / source / site / site_file))

for site, csv_info in tqdm(sites.items()):
    process_unmerged_site_data(site, csv_info)

100%|██████████| 386/386 [10:15<00:00,  1.59s/it]


In [5]:
# Get new metadata
meta_dfs = [pd.read_csv(INTERMEDIATE_DIR / source / 'site_data.csv') for source in os.listdir(INTERMEDIATE_DIR)]

def convert_time(timestep_string):
    return f'{timestep_string[0:4]}_{timestep_string[4:6]}_{timestep_string[6:8]}'

meta_df = pd.concat(meta_dfs, axis=0)[['SITE_ID', 'LOCATION_LAT', 'LOCATION_LON', 'LOCATION_ELEV', 'IGBP']].drop_duplicates('SITE_ID')
sites = os.listdir(OUTPUT_DIR)
meta_df = meta_df[meta_df['SITE_ID'].isin(sites)]
meta_df['TIME_INFO'] = pd.NA
for site in sites:
    subsets = os.listdir(os.path.join(OUTPUT_DIR, site))
    time_info = {s.split('_')[2]: [convert_time(s.split('_')[0]), convert_time(s.split('_')[1])] for s in subsets}
    filtered_rows = meta_df['SITE_ID'] == site
    meta_df.loc[filtered_rows, 'TIME_INFO'] = [time_info] * sum(filtered_rows)


if not os.path.exists(META_DIR):
    os.makedirs(META_DIR)
meta_df.to_csv(META_DIR / 'processed_site_meta.csv', index=False)